#### Imports/setup

In [1]:
%load_ext autoreload
%autoreload 2
# %reload_ext autoreload

In [2]:
import ee
import os
import pandas as pd
import geopandas as gpd
# import json
import geemap
import time
import functools
import os
import sys
# from datetime import datetime

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from modules.file_to_ceo import (
get_ceo_url, 
whisp_stats_shapefile_and_ceo_url
)

from parameters.config_runtime import (
    geo_id_column, 
    out_file_wide, 
    out_shapefile
)
from modules.agstack_to_gee import (
    start_agstack_session,
    get_agstack_token,
    geo_id_or_ids_to_feature_collection,
    shapefile_to_ee_feature_collection,
    register_fc_and_set_geo_id,
    register_fc_and_append_to_csv,
    add_geo_ids_to_feature_col_from_lookup_df,
    add_geo_ids_to_feature_col_from_lookup_csv
)

from modules.multiband_stats import get_stats,reformat_whisp_fc,get_stats_formatted

from parameters.config_asr_url_info import asset_registry_base, user_registry_base

from parameters.config_asr_credentials import email, password

from modules.utils import (
    collection_properties_to_df,
    remove_geometry_from_feature_collection,
    get_centroid#,
    # get_scale_from_image
)

def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out
   

Earth Engine has been initialized with the specified credentials.


##### Set up asset registry

In [3]:
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)

#### Inputs : with GEOIDS

##### Input: list of Geo ids

In [4]:
example_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907']

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids
#all_geo_ids= example_ids

In [5]:
#make into a feature collection
roi_geo_id = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

#### Get plot boundaries

In [6]:
roi = roi_geo_id
roi

### Whisp it

In [7]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
...stats processing started
...ee to df processing time: 9.811225652694702 seconds


### Display it

In [8]:
df

,Area_ha,Cocoa_ETH,Country,ESA_TC_2020,EUFO_2020,GFC_Loss_Year_2001,GFC_Loss_Year_2002,GFC_Loss_Year_2003,GFC_Loss_Year_2004,GFC_Loss_Year_2005,...,tmf_deg_2013,tmf_deg_2014,tmf_deg_2015,tmf_deg_2016,tmf_deg_2017,tmf_deg_2018,tmf_deg_2019,tmf_deg_2020,tmf_deg_2021,tmf_deg_2022
0,1.9,0,GHA,1.938262,0.362398,0.000000,0.061859,0.000000,0.000000,0.000000,...,0,0.000000,0,0.000000,0.035868,0.00000,0.00000,0.000000,0,0
1,16.6,0,GHA,16.602398,16.602398,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0,0.000000,0.000000,0.00000,0.00000,0.810621,0,0
2,31.2,0,IDN,29.087104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0.000000,0,0.000000,0.000000,0.00000,0.00000,0.000000,0,0
3,20.9,0,IDN,20.854253,5.841476,0.000000,0.000000,1.319234,1.289477,0.530416,...,0,0.089271,0,0.000000,0.000000,0.00000,0.00000,0.000000,0,0
4,8.3,0,CIV,6.402215,0.445641,0.744827,0.276781,0.000000,0.000000,0.000000,...,0,0.101874,0,0.039657,0.000000,0.04237,0.06873,0.000000,0,0
5,3.6,0,CIV,3.544055,1.484960,0.000000,0.000000,0.644486,0.000000,0.000000,...,0,0.000000,0,0.000000,0.000000,0.00000,0.00000,0.000000,0,0


#### Export it to CSV file

In [9]:
#export output csv
df.to_csv(path_or_buf=out_file_wide,header=True)

print ("output csv: ", out_file_wide)


output csv:  /home/sepal-user/whisp/whisp_output_table.csv
